<a href="https://colab.research.google.com/github/paolopdel/cat_dog_one_load/blob/this-code-loads-state_dict-of-code-cat_dog/cat_dog_one_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#import fc_model

In [0]:
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip

--2020-02-12 21:59:30--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.9.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.9.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580495262 (554M) [application/zip]
Saving to: ‘Cat_Dog_data.zip’

Cat_Dog_data.zip    100%[===================>] 553.60M  89.8MB/s    in 6.1s    

2020-02-12 21:59:36 (91.4 MB/s) - ‘Cat_Dog_data.zip’ saved [580495262/580495262]



In [0]:
!unzip Cat_Dog_data.zip

In [21]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
#train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
#test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
one_data = datasets.ImageFolder(data_dir + '/one', transform=train_transforms)
#create a directory under content/Cat_Dog_data ,  'one', create s ubdirectory 'cat'
#insert a jpeg photo of a cat
#trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
#testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
oneloader = torch.utils.data.DataLoader(one_data, batch_size=1)

print(one_data[0][1])
#one_data[0][1] = one_data[0][1]-1
#print(one_data[0][1])
#print(train_data[22000][1])
#print(test_data[2400][1])
#the Cat_Dog_data / train / test / cat directory is called 0
#the Cat_Dog_data / train / test / dog directory is called 1
#the Cat_Dog_data / one / cat directory is called 1

1


In [0]:
#torch.save(model.state_dict(), 'checkpoint.pth')

In [22]:
state_dict = torch.load('cat_dog_one.pth')
print(state_dict.keys())

odict_keys(['features.conv0.weight', 'features.norm0.weight', 'features.norm0.bias', 'features.norm0.running_mean', 'features.norm0.running_var', 'features.norm0.num_batches_tracked', 'features.denseblock1.denselayer1.norm1.weight', 'features.denseblock1.denselayer1.norm1.bias', 'features.denseblock1.denselayer1.norm1.running_mean', 'features.denseblock1.denselayer1.norm1.running_var', 'features.denseblock1.denselayer1.norm1.num_batches_tracked', 'features.denseblock1.denselayer1.conv1.weight', 'features.denseblock1.denselayer1.norm2.weight', 'features.denseblock1.denselayer1.norm2.bias', 'features.denseblock1.denselayer1.norm2.running_mean', 'features.denseblock1.denselayer1.norm2.running_var', 'features.denseblock1.denselayer1.norm2.num_batches_tracked', 'features.denseblock1.denselayer1.conv2.weight', 'features.denseblock1.denselayer2.norm1.weight', 'features.denseblock1.denselayer2.norm1.bias', 'features.denseblock1.denselayer2.norm1.running_mean', 'features.denseblock1.denselayer2

In [13]:
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [0]:
#model.load_state_dict(state_dict)

In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
model.load_state_dict(state_dict)
model.to(device);

In [24]:
#epochs = 1
#steps = 0
#running_loss = 0
#print_every = 5
#for epoch in range(epochs):
#    for inputs, labels in trainloader:
#        steps += 1
#        # Move input and label tensors to the default device
#        inputs, labels = inputs.to(device), labels.to(device)#

        #print(inputs.shape)
        #print(labels.shape)
#        optimizer.zero_grad()
        
#        logps = model.forward(inputs)
#        loss = criterion(logps, labels)
#        loss.backward()
#        optimizer.step()
        #print(inputs[0],labels[0])
        #print(loss.item())
#        running_loss += loss.item()

        
        #if steps % print_every == 0:
#        test_loss = 0
#        accuracy = 0
model.eval()
with torch.no_grad():
            for inputs, labels in oneloader:
                labels=labels-1
                #here as the Cat_Dog_data/one/cat directory is called 1, 
                # it must become 0 as the Cat_Dog_data/train/cat directory
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model.forward(inputs)
                batch_loss = criterion(logps, labels)
                    
                one_loss = batch_loss.item()
                    #one_loss = batch_loss
                    # Calculate accuracy
                    #ps = torch.exp(logps)
                    #top_p, top_class = ps.topk(1, dim=1)
                    #equals = top_class == labels.view(*top_class.shape)
                    #accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
#print(#f"Epoch {epoch+1}/{epochs}.. "
            #f"Train loss: {running_loss/print_every:.3f}.. "
            #f"Test loss: {test_loss/len(testloader):.3f}.. "
            #f"Test loss: {one_loss/len(testloader):.3f}.. "
            f"One loss: {one_loss:.3f}.. "
           # f"Test accuracy: {accuracy/len(oneloader):.3f}")

print(f"one_loss: {one_loss:.3f}" )        
#running_loss = 0
#model.train()
#print(inputs.shape)
# when I run the code, if I put the photo of 1 cat in the Cat_Dog_data / one / cat directory 
# I get one_loss close to zero.
# If instead I put a photo of a dog in the same directory I get one_loss greater than 1.

one_loss: 5.621
